# Load dim_order_warehouses từ Silver lên Gold

## Mục tiêu
- Load dữ liệu từ Silver.dim_order_warehouses → Gold.dim_order_warehouses
- Không cần biến đổi dữ liệu
- Chỉ cần copy và load vào Gold database

## Quy trình 3 bước
1. **Load from Silver** - Đọc dữ liệu từ Silver database
2. **Schema Definition** - Định nghĩa schema cho MySQL
3. **Load to Gold** - Load vào Gold database


## Bước 1: Import và Setup


In [1]:
# Import các thư viện cần thiết
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
from datetime import datetime

# Load biến môi trường từ file .env
load_dotenv()

# Lấy thông tin kết nối database
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_SILVER = os.getenv("DB_SILVER")
DB_GOLD = os.getenv("DB_GOLD")

# Tạo kết nối tới Silver và Gold database
silver_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_SILVER}")
gold_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_GOLD}")

print("Đã kết nối Silver và Gold database thành công")
print(f"Silver DB: {DB_SILVER}")
print(f"Gold DB: {DB_GOLD}")
print(f"Thời gian bắt đầu: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Đã kết nối Silver và Gold database thành công
Silver DB: winner_silver
Gold DB: winner_gold
Thời gian bắt đầu: 2025-10-17 23:42:17


## Bước 2: Load dữ liệu từ Silver


In [2]:
# Load dữ liệu dim_order_warehouses từ Silver database
print("=== LOADING DIM_ORDER_WAREHOUSES FROM SILVER ===")

# Load toàn bộ bảng dim_order_warehouses
warehouses_df = pd.read_sql("SELECT * FROM dim_order_warehouses", silver_engine)

# Hiển thị thông tin cơ bản về dataset
print(f"Shape: {warehouses_df.shape}")
print(f"Columns: {list(warehouses_df.columns)}")
print(f"Memory usage: {warehouses_df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

# Hiển thị sample data
print("\n=== SAMPLE DATA ===")
print(warehouses_df.head(5))

# Hiển thị data types
print("\n=== DATA TYPES ===")
print(warehouses_df.dtypes)

# Kiểm tra null values
print(f"\n=== NULL VALUES ===")
null_counts = warehouses_df.isnull().sum()
print(null_counts[null_counts > 0])


=== LOADING DIM_ORDER_WAREHOUSES FROM SILVER ===
Shape: (24, 7)
Columns: ['warehouse_id', 'warehouse_name', 'warehouse_address', 'warehouse_full_address', 'warehouse_phone_number', 'warehouse_commune_id', 'warehouse_province_id']
Memory usage: 0.01 MB

=== SAMPLE DATA ===
                           warehouse_id warehouse_name warehouse_address  \
0  a0b89882-3a6c-4167-8cad-37574a2be5f7   Kho mặc định              None   
1  2ff7f6ed-d985-49fd-9ba2-af8f78e7a5c5   Kho mặc định              None   
2  d4a2a199-55a6-4c01-920d-5616b88f03a1   Kho mặc định              None   
3  9146ec42-0429-4562-aae6-5144853f189f   Kho mặc định              None   
4  3e6994db-9491-4548-85d6-62f3ba784c82   Kho mặc định              None   

  warehouse_full_address warehouse_phone_number warehouse_commune_id  \
0                   None                   None                 None   
1                   None                   None                 None   
2                   None                   None       

In [4]:
# Xử lý dữ liệu warehouses
print("=== CLEANING WAREHOUSES DATA ===")

# Tạo bản copy để xử lý
warehouses_clean = warehouses_df.copy()

# 1. Chỉ giữ lại 3 cột đầu
print("1. Chỉ giữ lại 3 cột đầu...")
warehouses_clean = warehouses_clean.iloc[:, :3]
print(f"Columns sau khi giữ lại 3 cột đầu: {list(warehouses_clean.columns)}")

# 2. Tạo tên kho ngẫu nhiên (chỉ ở Hà Nội)
print("\n2. Tạo tên kho ngẫu nhiên ở Hà Nội...")
import random

# Danh sách tên kho ở Hà Nội
warehouse_names = [
    "Kho Hà Nội - Quận Ba Đình", "Kho Hà Nội - Quận Hoàn Kiếm", "Kho Hà Nội - Quận Tây Hồ",
    "Kho Hà Nội - Quận Long Biên", "Kho Hà Nội - Quận Cầu Giấy", "Kho Hà Nội - Quận Đống Đa",
    "Kho Hà Nội - Quận Hai Bà Trưng", "Kho Hà Nội - Quận Hoàng Mai", "Kho Hà Nội - Quận Thanh Xuân",
    "Kho Hà Nội - Quận Hà Đông", "Kho Hà Nội - Quận Nam Từ Liêm", "Kho Hà Nội - Quận Bắc Từ Liêm",
    "Kho Hà Nội - Quận Cầu Giấy", "Kho Hà Nội - Quận Đống Đa", "Kho Hà Nội - Quận Hai Bà Trưng",
    "Kho Hà Nội - Quận Hoàng Mai", "Kho Hà Nội - Quận Thanh Xuân", "Kho Hà Nội - Quận Hà Đông",
    "Kho Hà Nội - Quận Nam Từ Liêm", "Kho Hà Nội - Quận Bắc Từ Liêm", "Kho Hà Nội - Quận Ba Đình",
    "Kho Hà Nội - Quận Hoàn Kiếm", "Kho Hà Nội - Quận Tây Hồ", "Kho Hà Nội - Quận Long Biên"
]

# Tạo tên kho ngẫu nhiên cho mỗi record
random.seed(42)  # Để kết quả có thể tái tạo
warehouses_clean['warehouse_name'] = [random.choice(warehouse_names) for _ in range(len(warehouses_clean))]

print("Sample tên kho mới:")
print(warehouses_clean['warehouse_name'].value_counts().head(10))

# 3. Tạo địa chỉ ngẫu nhiên ở Hà Nội
print("\n3. Tạo địa chỉ ngẫu nhiên ở Hà Nội...")

# Danh sách địa chỉ mẫu ở Hà Nội
addresses = [
    "Số 1, Phố Huế, Phường Phố Huế, Quận Hai Bà Trưng, Hà Nội",
    "Số 15, Đường Láng, Phường Láng Thượng, Quận Đống Đa, Hà Nội",
    "Số 25, Phố Hàng Bạc, Phường Hàng Bạc, Quận Hoàn Kiếm, Hà Nội",
    "Số 35, Đường Giải Phóng, Phường Đồng Tâm, Quận Hai Bà Trưng, Hà Nội",
    "Số 45, Phố Hàng Đào, Phường Hàng Đào, Quận Hoàn Kiếm, Hà Nội",
    "Số 55, Đường Cầu Giấy, Phường Dịch Vọng, Quận Cầu Giấy, Hà Nội",
    "Số 65, Phố Hàng Mã, Phường Hàng Mã, Quận Hoàn Kiếm, Hà Nội",
    "Số 75, Đường Láng Hạ, Phường Láng Hạ, Quận Đống Đa, Hà Nội",
    "Số 85, Phố Hàng Bồ, Phường Hàng Bồ, Quận Hoàn Kiếm, Hà Nội",
    "Số 95, Đường Kim Mã, Phường Kim Mã, Quận Ba Đình, Hà Nội",
    "Số 105, Phố Hàng Gai, Phường Hàng Gai, Quận Hoàn Kiếm, Hà Nội",
    "Số 115, Đường Nguyễn Chí Thanh, Phường Nguyễn Chí Thanh, Quận Ba Đình, Hà Nội",
    "Số 125, Phố Hàng Buồm, Phường Hàng Buồm, Quận Hoàn Kiếm, Hà Nội",
    "Số 135, Đường Lê Duẩn, Phường Lê Duẩn, Quận Ba Đình, Hà Nội",
    "Số 145, Phố Hàng Chiếu, Phường Hàng Chiếu, Quận Hoàn Kiếm, Hà Nội",
    "Số 155, Đường Trần Duy Hưng, Phường Trần Duy Hưng, Quận Cầu Giấy, Hà Nội",
    "Số 165, Phố Hàng Mắm, Phường Hàng Mắm, Quận Hoàn Kiếm, Hà Nội",
    "Số 175, Đường Phạm Văn Đồng, Phường Phạm Văn Đồng, Quận Cầu Giấy, Hà Nội",
    "Số 185, Phố Hàng Đường, Phường Hàng Đường, Quận Hoàn Kiếm, Hà Nội",
    "Số 195, Đường Láng, Phường Láng, Quận Đống Đa, Hà Nội",
    "Số 205, Phố Hàng Bông, Phường Hàng Bông, Quận Hoàn Kiếm, Hà Nội",
    "Số 215, Đường Nguyễn Trãi, Phường Nguyễn Trãi, Quận Thanh Xuân, Hà Nội",
    "Số 225, Phố Hàng Bè, Phường Hàng Bè, Quận Hoàn Kiếm, Hà Nội",
    "Số 235, Đường Lê Văn Lương, Phường Lê Văn Lương, Quận Thanh Xuân, Hà Nội"
]

# Tạo địa chỉ ngẫu nhiên
warehouses_clean['warehouse_address'] = [random.choice(addresses) for _ in range(len(warehouses_clean))]

print("Sample địa chỉ mới:")
print(warehouses_clean['warehouse_address'].head(5).tolist())

# Hiển thị kết quả
print(f"\n=== KẾT QUẢ SAU KHI CLEANING ===")
print(f"Shape: {warehouses_clean.shape}")
print(f"Columns: {list(warehouses_clean.columns)}")
print(f"Null values: {warehouses_clean.isnull().sum().sum()}")

print(f"\nSample data sau khi cleaning:")
print(warehouses_clean.head(10))

=== CLEANING WAREHOUSES DATA ===
1. Chỉ giữ lại 3 cột đầu...
Columns sau khi giữ lại 3 cột đầu: ['warehouse_id', 'warehouse_name', 'warehouse_address']

2. Tạo tên kho ngẫu nhiên ở Hà Nội...
Sample tên kho mới:
warehouse_name
Kho Hà Nội - Quận Long Biên      5
Kho Hà Nội - Quận Ba Đình        4
Kho Hà Nội - Quận Hoàng Mai      3
Kho Hà Nội - Quận Thanh Xuân     2
Kho Hà Nội - Quận Hoàn Kiếm      2
Kho Hà Nội - Quận Tây Hồ         2
Kho Hà Nội - Quận Cầu Giấy       1
Kho Hà Nội - Quận Hà Đông        1
Kho Hà Nội - Quận Nam Từ Liêm    1
Kho Hà Nội - Quận Đống Đa        1
Name: count, dtype: int64

3. Tạo địa chỉ ngẫu nhiên ở Hà Nội...
Sample địa chỉ mới:
['Số 175, Đường Phạm Văn Đồng, Phường Phạm Văn Đồng, Quận Cầu Giấy, Hà Nội', 'Số 65, Phố Hàng Mã, Phường Hàng Mã, Quận Hoàn Kiếm, Hà Nội', 'Số 225, Phố Hàng Bè, Phường Hàng Bè, Quận Hoàn Kiếm, Hà Nội', 'Số 205, Phố Hàng Bông, Phường Hàng Bông, Quận Hoàn Kiếm, Hà Nội', 'Số 225, Phố Hàng Bè, Phường Hàng Bè, Quận Hoàn Kiếm, Hà Nội']

=== KẾ

In [7]:
# Load vào Gold Database
print("=== LOADING TO GOLD DATABASE ===")

from sqlalchemy.types import String, Text

# Định nghĩa schema mapping cho MySQL
dtype_mapping = {
    'warehouse_id': String(100),      # UUID string
    'warehouse_name': String(255),    # Tên kho
    'warehouse_address': Text()       # Địa chỉ đầy đủ
}

print(f"Schema mapping defined for {len(dtype_mapping)} columns")

# Load vào Gold database
table_name = "dim_order_warehouses"

try:
    warehouses_clean.to_sql(
        table_name,
        con=gold_engine,
        if_exists="replace",  # Ghi đè dữ liệu cũ
        index=False,
        dtype=dtype_mapping
    )
    
    print(f"Đã load {warehouses_clean.shape[0]} records vào Gold: {table_name}")
    
    # Verify load
    verification_query = f"SELECT COUNT(*) as count FROM {table_name}"
    verification_result = pd.read_sql(verification_query, gold_engine)
    loaded_count = verification_result['count'].iloc[0]
    
    print(f"Verification: {loaded_count} records trong database")
    print(f"Schema: {len(dtype_mapping)} columns với explicit typing")
    
    # Sample data từ database
    sample_query = f"SELECT * FROM {table_name} LIMIT 5"
    sample_db = pd.read_sql(sample_query, gold_engine)
    print(f"\nSample data từ Gold database:\n{sample_db}")
    
    # Thống kê tên kho
    stats_query = f"SELECT warehouse_name, COUNT(*) as count FROM {table_name} GROUP BY warehouse_name ORDER BY count DESC"
    stats_result = pd.read_sql(stats_query, gold_engine)
    print(f"\nThống kê phân bố kho:")
    print(stats_result)
    
except Exception as e:
    print(f"Error loading to Gold: {str(e)}")
    raise

print(f"\nLoad dim_order_warehouses completed successfully!")
print(f"Ready for EDA and analytics")

=== LOADING TO GOLD DATABASE ===
Schema mapping defined for 3 columns
Đã load 24 records vào Gold: dim_order_warehouses
Verification: 24 records trong database
Schema: 3 columns với explicit typing

Sample data từ Gold database:
                           warehouse_id                warehouse_name  \
0  a0b89882-3a6c-4167-8cad-37574a2be5f7     Kho Hà Nội - Quận Ba Đình   
1  2ff7f6ed-d985-49fd-9ba2-af8f78e7a5c5   Kho Hà Nội - Quận Long Biên   
2  d4a2a199-55a6-4c01-920d-5616b88f03a1     Kho Hà Nội - Quận Ba Đình   
3  9146ec42-0429-4562-aae6-5144853f189f   Kho Hà Nội - Quận Long Biên   
4  3e6994db-9491-4548-85d6-62f3ba784c82  Kho Hà Nội - Quận Thanh Xuân   

                                   warehouse_address  
0  Số 175, Đường Phạm Văn Đồng, Phường Phạm Văn Đ...  
1  Số 65, Phố Hàng Mã, Phường Hàng Mã, Quận Hoàn ...  
2  Số 225, Phố Hàng Bè, Phường Hàng Bè, Quận Hoàn...  
3  Số 205, Phố Hàng Bông, Phường Hàng Bông, Quận ...  
4  Số 225, Phố Hàng Bè, Phường Hàng Bè, Quận Hoàn...  

T

In [8]:
# Tạo Data Dictionary cho Gold
print("=== CREATING GOLD DATA DICTIONARY ===")

def get_business_meaning_vietnamese(column_name):
    """Get business meaning for each column in Vietnamese"""
    business_meanings = {
        # Định danh & Cơ bản
        "warehouse_id": "Mã định danh kho duy nhất (khóa chính)",
        "warehouse_name": "Tên kho hàng (đã được clean và đa dạng hóa)",
        "warehouse_address": "Địa chỉ kho hàng đầy đủ (đã được tạo ngẫu nhiên)"
    }
    return business_meanings.get(column_name, "Chưa định nghĩa ý nghĩa business")

# Tạo Data Dictionary
dict_data = []
for col in warehouses_clean.columns:
    col_info = {
        "table_name": "dim_order_warehouses",
        "column_name": col,
        "dtype": str(warehouses_clean[col].dtype),
        "sql_type": str(dtype_mapping.get(col, "Not defined")),
        "null_count": warehouses_clean[col].isnull().sum(),
        "null_pct": round(warehouses_clean[col].isnull().mean() * 100, 2),
        "unique_count": warehouses_clean[col].nunique(),
        "sample_values": str(warehouses_clean[col].dropna().unique()[:3].tolist()),
        "business_meaning": get_business_meaning_vietnamese(col),
        "extraction_date": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    dict_data.append(col_info)

data_dictionary = pd.DataFrame(dict_data)

# Hiển thị Data Dictionary
print(f"Generated Data Dictionary for {len(data_dictionary)} columns")
print("\n=== DATA DICTIONARY ===")
print(data_dictionary)

# Lưu vào file Excel
excel_path = "Technical_Document/Gold_dictionary.xlsx"

try:
    from openpyxl import load_workbook
    
    # Kiểm tra file Excel có tồn tại không
    try:
        # Load workbook hiện tại
        wb = load_workbook(excel_path)
        
        # Lấy sheet đầu tiên
        ws = wb.active
        
        # Kiểm tra xem có dữ liệu cũ không
        if ws.max_row > 1:
            print(f"Found existing data in {excel_path}, appending new data...")
        else:
            print(f"File {excel_path} exists but is empty, adding header and data...")
            
    except FileNotFoundError:
        print(f"File {excel_path} not found, creating new file...")
        wb = None
    except Exception as e:
        print(f"Error loading {excel_path}: {str(e)}, creating new file...")
        wb = None
    
    if wb is None:
        # Tạo file mới với header
        data_dictionary.to_excel(excel_path, index=False, sheet_name='Gold_Dictionary')
        print(f"✅ Created new file: {excel_path}")
    else:
        # Append vào file hiện tại
        from openpyxl.utils.dataframe import dataframe_to_rows
        
        # Tìm dòng cuối cùng có dữ liệu
        last_row = ws.max_row
        
        # Thêm dữ liệu mới từ dòng tiếp theo
        for r in dataframe_to_rows(data_dictionary, index=False, header=False):
            last_row += 1
            for c_idx, value in enumerate(r, 1):
                ws.cell(row=last_row, column=c_idx, value=value)
        
        # Lưu file
        wb.save(excel_path)
        print(f"✅ Appended {len(data_dictionary)} rows to: {excel_path}")
    
except Exception as e:
    print(f"❌ Error appending to Data Dictionary: {str(e)}")
    # Fallback: tạo file mới
    try:
        data_dictionary.to_excel(excel_path, index=False)
        print(f"✅ Created new file as fallback: {excel_path}")
    except Exception as e2:
        print(f"❌ Error creating fallback file: {str(e2)}")

# Summary Report
print(f"\n=== GOLD DICTIONARY SUMMARY ===")
print(f"Table: dim_order_warehouses")
print(f"Total columns: {len(data_dictionary)}")
print(f"Data Dictionary: {excel_path}")
print(f"Created at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n=== SAMPLE DICTIONARY ENTRIES ===")
sample_dict = data_dictionary[['column_name', 'dtype', 'business_meaning']].head(5)
print(sample_dict)

=== CREATING GOLD DATA DICTIONARY ===
Generated Data Dictionary for 3 columns

=== DATA DICTIONARY ===
             table_name        column_name   dtype      sql_type  null_count  \
0  dim_order_warehouses       warehouse_id  object  VARCHAR(100)           0   
1  dim_order_warehouses     warehouse_name  object  VARCHAR(255)           0   
2  dim_order_warehouses  warehouse_address  object          TEXT           0   

   null_pct  unique_count                                      sample_values  \
0       0.0            24  ['a0b89882-3a6c-4167-8cad-37574a2be5f7', '2ff7...   
1       0.0            12  ['Kho Hà Nội - Quận Ba Đình', 'Kho Hà Nội - Qu...   
2       0.0            16  ['Số 175, Đường Phạm Văn Đồng, Phường Phạm Văn...   

                                   business_meaning      extraction_date  
0            Mã định danh kho duy nhất (khóa chính)  2025-10-17 23:51:07  
1       Tên kho hàng (đã được clean và đa dạng hóa)  2025-10-17 23:51:07  
2  Địa chỉ kho hàng đầy đủ (đã